In January 2024 I tried the rowing machine, also known as the ergometer. Basically rowing indoors without ever moving a bit. Because it's a very good body workout and less prone to injury. And the sauna afterwards is a great reward.

So this got a little out of control, in a good way. In total I rowed 1000 km (or 1 million meters).

In [6]:
#| echo: false
import pandas as pd
import numpy as np
import altair as alt
from datetime import timedelta, datetime

from fh_altair import altair2fasthtml
from fasthtml.common import *
from fh_altair import altair_headers    

ENV = 'Dev'
live = False if ENV == 'Prod' else True
rowtypes = ['Sprint', 'Endurance', 'Long distance']

# alt.renderers.enable("jupyter")


In [7]:
#| include: false

def descriptives(rows, rowtype):
    selected_rows = [row for row in rows() if row.type == rowtype]
    if len(selected_rows)==0: return 'no rows'
    distance = sum(row.distance for row in selected_rows) // 1000
    best = min(row.time for row in selected_rows)
    bestdate = next(row.date for row in selected_rows if row.time == best)
    amount = sum(1 for row in selected_rows)
    return f'Amount of rows {amount} total distance {distance} best_time {best} best_date {bestdate}'

def total_distance(rows):
    return sum(row.distance for row in rows()) //1000
def render(row):
    tid = f"row-{row.id}"
    toggle = A("Select ", hx_get=f"/toggle/{row.id}", target_id=tid)
    delete = A(
        "Delete ", hx_delete=f"/toggle/{row.id}", target_id=tid, hx_swap="outerHTML"
    )
    return Li(toggle, delete, display(row) + (" ✅" if row.done else " 🔎"), id=tid)

def str2time(dis, time):
    args = time.split(":")
    if int(dis) >= 10000:
        mult = [36000,600,10]
    else:
        mult = [600,10,1]
    # return 1
    return sum(int(arg) * mult for arg, mult in zip(args, mult))
        
def display(row):
    res = 0
    
    if row.distance >= 10000:
        mult = [36000,600,10]
    else:
        mult = [600,10,1]
    mult = list(reversed(mult))
    val = row.time
    res = []
    while val:
        add, val = divmod(val, mult.pop())
        res.append(str(add))
    return f'{row.type, row.date, row.distance, row.time, row.time_str, row.sprint_amount}'
    



app, rt, rows, Row = fast_app(
    "data//rowsss.db", live=live, 
        id=int, 
        date=str, 
        type=str, 
        distance=int, 
        time=int,
        done=bool, 
        remark= str, 
        break_rest   = int,
        sprint_amount = int,
        sprint_fastest = int,
        last_interval = str,
        endurance_breakmins = int,
        time_str = str,
    pk="id", render=render, hdrs = altair_headers
)
# rows = actual table
# row = object type within table



def mk_form():
    # return (Input(placeholder="type here", id="title", test = 'bla',hx_swap_oob="true"), Input(placeholder="type here", id="title", hx_swap_oob="true"))

    return Fieldset(
            Label('Date', Input(name="date", type='date')),
            Label("Sprint", Input(name="type", type='radio', value='Sprint')),
            Label("Endurance", Input(name="type", type='radio', value='Endurance')),
            Label("Long distance", Input(name="type", type='radio', value='Long distance')),
            Label("Distance", Input(name="distance")),
            Label("Time", Input(name="time")),
            Label("Remark", Input(name="remark")),
            Label("Break rest", Input(name="break_rest")),
            Label("# Interval", Input(name="sprint_amount")),
            Label("Interval fastest", Input(name="sprint_fastest")),
            Label("Last interval", Input(name="last_interval")),
        )

@rt("/")
def get():
    return Body(
        Header(navigation_bar(list(nav_items.keys()))),
        Div(enter(), id="page-content"),
    )

@app.route("/enter", methods=["get"])
def enter():
       
    frm = Form(
        Group(mk_form(), Button("Add row")),
        hx_post="/",
        target_id="row-list",
        hx_swap="beforeend",
    )
    # rows.insert(row(title='first', done=False))
    return Titled(
        "rows",
        Card(
            Ul(*list(reversed(rows())), id="row-list"),
            header=frm,
            #                       ),
        ),
        Div(P('total distance', total_distance(rows), hx_get="/")),
        Ul(*[Div(P(descriptives(rows, rowtype), hx_get="/")) for rowtype in rowtypes]),
    )


nav_items = {'New training entry':'enter',
            'Total distance': 'cumdis',
             'Split times': 'endurancesplits',
             'Top 500m':'fastest500m',
             'Top 2k':'fastest2k',
             'Top 5k':'fastest5k',
             'Top 10k':'fastest10k',
             }
def navigation_bar(navigation_items: list[str]):
    return Nav(
        Ul(
            *[
                Button(item, hx_get=f"/{nav_items[item]}", hx_target="#page-content")
                for item in navigation_items
            ]
        ),
    )   

@rt("/")
def post(row:dict):
    if row['type'] != 'Sprint':    
        row['sprint_amount'] = 0
        row['sprint_fastest'] = 0
    row['time_str'] = row['time']
    row['time'] = str2time(row['distance'], row['time'])
    
        
    # row['distance'] = 2
    # print(row.__dict__.__repr__())
    # print(row, Row(row).__dict__.repr())
    return rows.insert(row)


@rt("/toggle/{tid}")
def get(tid: int):
    row = rows[tid]
    row.time = row.time // 7
    # row.done = not row.done
    return rows.update(row)


In [8]:
#| include: false
type_trans = {'Sprint':'7x500m sprints','Endurance':'Endurance <10km', 'Long distance':'Long distance >=10km'}
def change_graph(func):
    def wrap(*args, **kwargs):
        if 'height' in kwargs:
            return func(*args, **kwargs)
            height = kwargs.pop('height')
            width = kwargs.pop('width')
            return func(*args, **kwargs).properties(height=height, width=width)
        else: 
            return func(*args, **kwargs).properties(height=800, width = 100)
    return wrap


@change_graph
def generate_cumul_distance(online=True, height=300, width='600'):
    # alt.renderers.enable('vegafusion')
    
    dates = [row.date for row in rows()]
    types = [type_trans[row.type] for row in rows()]
    types = [t for _, t in sorted(zip(dates, types))]
    distances = [row.distance/1000 if row.type != 'Sprint' else row.distance /1000 * int(row.sprint_amount) for row in rows()]
    distances = [dis for _, dis in sorted(zip(dates, distances))]
    dates.sort()
    cumul_distances = distances.copy()
    for i in range(1, len(distances)):
        cumul_distances[i] += cumul_distances[i-1]

    pltr = pd.DataFrame({'Date': dates, 'Cumulative distance (km)': cumul_distances, 'Distance': distances, 'Training type':types})
    chart = alt.Chart(pltr).mark_line(
        strokeWidth=alt.param('1'), color='lightgrey'
        ).encode(
            alt.X('Date:T', title=None, axis=alt.Axis(
                    format='%b',
                    ), 
                scale=alt.Scale(
                    domainMin=alt.DateTime(year=2024, month=1, day=1),
                    )
                  ), 
                
            alt.Y('Cumulative distance (km):Q', axis=alt.Axis(tickCount=5),
                  ), 
            tooltip=[
                alt.Tooltip('Date:T', title='Date', format='%d %b %Y'),
                alt.Tooltip('Distance:Q', title='Distance (km)'), 
                alt.Tooltip('Cumulative distance (km):Q', title='Cumulative distance (km)'), 
                ] 
        ).properties(width='container', height=height)
    
    points = chart.mark_point(filled=True,size=38).encode(
            alt.X('Date:T', title=None,  
                scale=alt.Scale(
                    domainMin=alt.DateTime(year=2024, month=1, day=1),
                    )
                  ), 
                    
        
        color=alt.Color('Training type:N').legend(orient='top-left').scale(scheme='set1'),
        
    )
    
    df = pd.DataFrame({
        'x': ["1-1-2024", "7-1-2024", "11-1-2024"],  # Relative x position
        'y': [0,600,0],  # Relative y position
        'text': ['', 'The road to 1 million meters...', '']
    })
    # Create a text mark for the title
    text = alt.Chart(df).mark_text(
        align='center',
        baseline='middle',
        fontSize=25,
        angle=338 # Set the desired angle for the text
    ).encode(
        x = alt.X('x:T'),
        y=alt.Y("y:Q", title='Cumulative distance (km)' ),
        text='text:N'
    )

    # text="Origin[0]:N",
    
    
    return altair2fasthtml((chart+points+text).configure_axis()) if online else (text+
                                                                                 chart+points)

def generate_fastest_endurance(dis=None, online=True, height=300):
    
    if dis:
        types = [row.type for row in rows() if row.distance == dis]
        times = [row.time/10 for row in rows() if row.distance == dis]
        times = [t*1000 for t in times]
        distances = [row.distance for row in rows() if row.distance == dis]
        dates = [row.date for row in rows() if row.distance == dis]
        splits = [t/dis*500 for t in times]
        distances = [d/1000 for d in distances]
    else:
        types = [row.type for row in rows()]
        times = [row.time/10 for row in rows()]
        times = [t*1000 for t in times]
        distances = [row.distance for row in rows()]
        dates = [row.date for row in rows()]
        splits = [t/d*500 for t,d in zip(times, distances)]
        distances = [d/1000 for d in distances]
    pltr = pd.DataFrame({'Date': dates, 'Time': times, 'Split': splits, 'Type':types, 'Distance (km)':distances})
    pltr['Time'] = pd.to_datetime(pltr['Time']/1000, unit='s').dt.strftime('%H:%M:%S')

    
    times.sort()
    splits.sort()
    gold = f"datum.Time == '{times[0]}'"
    silver = f"datum.Time == '{times[1]}'"
    bronze = f"datum.Time == '{times[2]}'"
    
    if dis:
        chart = alt.Chart(pltr).mark_point(filled=True).encode(
            alt.X('Date:T', axis=alt.Axis(format='%b')), 
            alt.Y('Split:T', axis=alt.Axis(format="%M:%S:%L",tickCount=2)),
            color=alt.condition(gold, alt.value('gold'), alt.Color('Type:N', legend=None)),
            ).properties(width='container', height=height)

        chart2 = alt.Chart(pltr).mark_point(filled=True).encode(
                alt.X('Date:T', axis=alt.Axis(format='%b')), 
                alt.Y('Time:T', axis=alt.Axis(format="%M:%S:%L",tickCount=2)),
                color=alt.condition(gold, alt.value('gold'), alt.Color('Type:N', legend=None)),
                tooltip=[
                alt.Tooltip('Date:T', title='Date', format='%d %b %Y'),
                alt.Tooltip('Distance (km):Q', title='Distance (km)'), 
                alt.Tooltip('Time:N', title='Time (hr:min:sec)'), 
                alt.Tooltip('Split:T', title='500m split (min:sec)', format='%M:%S'),
                ] 
                ).properties(width='container', height=height)
    else:
        chart = alt.Chart(pltr).mark_point(filled=True).encode(
            alt.X('Date:T', axis=alt.Axis(
                    format='%b', 
                    labelOverlap=False, 
                    # tickCount=6, # altair can determine this better based on the screen width
                    # labelExpr='datum.label[0]', to only display J instead of Jan. But its confusing
                    ), 
                title=None,
                scale=alt.Scale(
                    domainMin=alt.DateTime(year=2024, month=1, day=1),
                    )
                ), 
            alt.Y('Split:T', axis=alt.Axis(
                format="%M:%S",)
                , title='Split time, lower is faster (min:sec)'),
            size='Distance (km):Q',
            # color=alt.condition(gold, alt.value('gold'), 'Type:N'),
            color=alt.Color('Distance (km):Q', scale=alt.Scale(scheme='turbo')).legend(orient='top'),  
            tooltip=[
                alt.Tooltip('Date:T', title='Date', format='%d %b %Y'),
                alt.Tooltip('Distance (km):Q', title='Distance (km)'), 
                alt.Tooltip('Time:N', title='Time (hr:min:sec)'), 
                alt.Tooltip('Split:T', title='500m split, (min:sec)', format='%M:%S'),
                ] 
            ).properties(width='container', height=height)
    best = (datetime(1,1,1) + timedelta(seconds=times[0]/1000)).strftime('%M:%S')
    bestsplit = (datetime(1,1,1) + timedelta(seconds=splits[0]/1000)).strftime('%M:%S:%f')[:-5]

    if dis:
        title = f'{dis}m Personal Best {best} (split {bestsplit}) {len(times)} sessions'
        return altair2fasthtml((chart+chart2).resolve_scale(y='independent').properties(title=title))
    else:
        title = {'text': f'Rowing speed for each training',
                 'subtitle': ['Average 500m split time, excluding breaks'], 
                 'subtitleColor': 'gray', # Optional: Set the subtitle color 
                 }
        return altair2fasthtml(chart.properties(title=title)) if online else chart.properties(title=title)

def generate_splits(t, height=400, width=200):
    # height, width = 400, 200
    distances = [row.distance for row in rows() if row.type ==t]
    times = [row.time/10 for row in rows() if row.type == t]
    dates = [row.date for row in rows() if row.type == t]
    splits = [round(t/d*500)*1000 for t, d in zip(times, distances)]
    print(len(distances), len(times), len(dates), len(splits))
    
    pltr = pd.DataFrame({'x': dates, 'y': splits, 'distance': distances})
    chart = alt.Chart(pltr).mark_point().encode(
        alt.X('x:T', axis=alt.Axis(format='%b')), 
        alt.Y('y:T', axis=alt.Axis(format="%M:%S", formatType='time')), 
        tooltip=['distance'],
        
        ).properties(width=width, height=height)
    
    pltr = pd.DataFrame({'x': distances, 'y': splits, 'date':dates})
    chart2 = alt.Chart(pltr).mark_point().encode(alt.X('x:Q'), alt.Y('y:T', axis=alt.Axis(format="%M:%S", formatType='time')), tooltip=['date', 'x', 'y']).properties(width='width', height=height)
    
    return [altair2fasthtml(chart), altair2fasthtml(chart2)]


@rt("/cumdis")
def get():
    return generate_cumul_distance()

@rt("/fastest500m")
def get():
    return generate_fastest_endurance(500)

@rt("/fastest2k")
def get():
    return generate_fastest_endurance(2000)

@rt("/fastest5k")
def get():
    return generate_fastest_endurance(5000)
    
@rt("/fastest10k")
def get():
    return generate_fastest_endurance(10000)

@rt("/endurancesplits")
def get():
    return generate_fastest_endurance()


@rt("/toggle/{tid}")
def delete(tid: int):
    rows.delete(tid)



In [9]:
#| echo: false

generate_cumul_distance(online=False, height=300, width=500)

alt.LayerChart(...)

&nbsp; 

In April I started doing longer distances at a slower pace. End of September the goal to reach 1 million meters in a year materialized. 

So that was tough but it was reached! Including a marathon December 14th and 100km in a week. 

&nbsp; 

In [10]:
#| echo: false
generate_fastest_endurance(None, online=False, height=400)

alt.Chart(...)

&nbsp; 

Here you see the split times. The small dots represent the short rows. Of course the average split times are faster compared to a longer training where you go slower.

You can see that at the end of the year I was steadily going slower. Of course I was also rowing longer, but maybe got a bit fatigued.

Anyway, it was fun!